In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Input, GlobalAveragePooling2D
from glob import glob


Using TensorFlow backend.


In [2]:
train = '/home/hasan/Desktop/DATA_SET/chest_xray/train/'
test = '/home/hasan/Desktop/DATA_SET/chest_xray/test/'
validation = '/home/hasan/Desktop/DATA_SET/chest_xray/val/'

In [3]:
#for finding number of folder in train folder
n_folders = glob('/home/hasan/Desktop/DATA_SET/chest_xray/train/*')

# Defining Model

In [4]:
base_model = VGG16(input_shape = (224,224,3), weights='imagenet', include_top=False)

In [5]:
x = base_model.output
x = Flatten()(x)
x = Dense(350, activation='relu')(x)
x = Dense(300, activation='relu')(x)
prediction  = Dense(len(n_folders), activation='softmax')(x) 

model = Model(inputs=base_model.input , outputs=prediction)

In [6]:
#freeze all convolutional vgg16 layer
for layer in base_model.layers:
    layer.trainable = False

# Compiling The Model

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Summary Of The Model

In [8]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

# Defining ImageDataGenerator

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=.2,
                                   zoom_range=.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_data_generator = train_datagen.flow_from_directory('/home/hasan/Desktop/DATA_SET/chest_xray/train/',
                                                  target_size=(224,224),
                                                  batch_size=32,
                                                  class_mode='categorical')

Found 5216 images belonging to 2 classes.


In [11]:
test_data_generator = test_datagen.flow_from_directory('/home/hasan/Desktop/DATA_SET/chest_xray/test/',
                                                      target_size=(224,224),
                                                      batch_size=32,
                                                      class_mode='categorical'
                                                      )

Found 624 images belonging to 2 classes.


# Training The Model

In [13]:
model_fit = model.fit_generator(generator=train_data_generator, 
              steps_per_epoch=len(train_data_generator),
              epochs=5,   
              validation_data=test_data_generator, 
              validation_steps=len(test_data_generator),  
              )

Epoch 1/5
163/163 [==============================] - 1250s 8s/step - loss: 0.1750 - accuracy: 0.9325 - val_loss: 0.8989 - val_accuracy: 0.8670
Epoch 2/5
163/163 [==============================] - 1186s 7s/step - loss: 0.1191 - accuracy: 0.9548 - val_loss: 0.7470 - val_accuracy: 0.8157
Epoch 3/5
163/163 [==============================] - 1179s 7s/step - loss: 0.1013 - accuracy: 0.9615 - val_loss: 0.0752 - val_accuracy: 0.9199
Epoch 4/5
163/163 [==============================] - 1181s 7s/step - loss: 0.0848 - accuracy: 0.9674 - val_loss: 0.1164 - val_accuracy: 0.8510
Epoch 5/5
163/163 [==============================] - 1172s 7s/step - loss: 0.0858 - accuracy: 0.9657 - val_loss: 0.3002 - val_accuracy: 0.8942


# Accuracy and Loss of Testing and Training Data

In [ ]:
val_accuracy = model_fit.history['val_accuracy']

loss = model_fit.history['loss']
val_loss = model_fit.history['val_loss']

epochs = range(len(val_accuracy))
plt.plot(epochs, val_accuracy, 'go', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'go', label='Training loss')
plt.plot(epochs, val_loss, 'g', label='validation loss')
plt.title('training and validation loss')
plt.legend()
plt.show()

I have took the number of epochs=5, for this graph looks like that. 
If epochs would 20 our graphs would looks like very smooth

I could also use VGG19 Application alternative of VGG16